<a href="https://colab.research.google.com/github/AgomferAustral/DMA-Caras/blob/main/Eigenfaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# conexion al Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
# Importacion de librerias
#!pip install --upgrade mediapipe


In [28]:
import os
import cv2
import sys
import numpy as np

In [29]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [30]:
#!pip install --upgrade numpy
#!pip install --upgrade scipy

In [31]:
# Definicion de ruta de conexion
ruta_entrada = '/content/drive/MyDrive/Eigenfaces'
ruta_salida = '/content/drive/MyDrive/Eigenfaces70x70'

In [32]:
# prompt: verificar si la carpeta en "ruta_salida" existe. Si no existe, crearla

if not os.path.exists(ruta_salida):
  os.makedirs(ruta_salida)


In [37]:
def procesar_archivos_en_carpetas(ruta_principal,ruta_final, img_size=(70, 70)):
  """Recorre las carpetas dentro de la ruta principal y procesa los archivos.

  Args:
    ruta_principal: La ruta de la carpeta principal.
  """
  print(f"Ruta principal: {ruta_principal}")
  for carpeta_actual, _, archivos in os.walk(ruta_principal):
    print(f"Carpeta actual: {carpeta_actual}")
    for archivo in archivos:
      print(f"Archivo: {archivo}")
      ruta_completa = os.path.join(carpeta_actual, archivo)
      try:

            img = cv2.imread(ruta_completa, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"⚠️ No se pudo leer: {ruta_completa}")
                continue

            # Detectar rostros
            faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=10, minSize=(70, 70))

            for i, (x, y, w, h) in enumerate(faces):
                face = img[y:y+h, x:x+w]  # Recortar rostro
                face_resized = cv2.resize(face, img_size)  # Redimensionar a 70x70

                # Crear carpeta de salida manteniendo la estructura original
                relative_path = os.path.relpath(carpeta_actual, ruta_entrada)
                output_folder = os.path.join(ruta_salida, relative_path)
                os.makedirs(output_folder, exist_ok=True)

                # Guardar rostro procesado
                output_path = os.path.join(output_folder, f"{os.path.splitext(archivo)[0]}_face{i}.jpg")
                cv2.imwrite(output_path, face_resized)
                print(f"✅ Guardado: {output_path}")


      except Exception as e:
          print(f"❌ Error procesando {ruta_completa}: {e}")







      except Exception as e:
        print(f"Error al procesar el archivo {ruta_completa}: {e}")

# Ejemplo de uso:
print("Procesando archivos")
procesar_archivos_en_carpetas(ruta_entrada, ruta_salida, img_size=(70, 70))
print("Proceso inicial terminado ...")

Procesando archivos
Ruta principal: /content/drive/MyDrive/Eigenfaces
Carpeta actual: /content/drive/MyDrive/Eigenfaces
Carpeta actual: /content/drive/MyDrive/Eigenfaces/Noelia R
Archivo: 1742907849929.jpg
✅ Guardado: /content/drive/MyDrive/Eigenfaces70x70/Noelia R/1742907849929_face0.jpg
Archivo: 20250321_170637.jpg
✅ Guardado: /content/drive/MyDrive/Eigenfaces70x70/Noelia R/20250321_170637_face0.jpg
Archivo: IMG_4185.JPG
✅ Guardado: /content/drive/MyDrive/Eigenfaces70x70/Noelia R/IMG_4185_face0.jpg
Archivo: IMG_20250321_170629085.jpg
✅ Guardado: /content/drive/MyDrive/Eigenfaces70x70/Noelia R/IMG_20250321_170629085_face0.jpg
Archivo: IMG_6804.jpeg
✅ Guardado: /content/drive/MyDrive/Eigenfaces70x70/Noelia R/IMG_6804_face0.jpg
Archivo: 1742907849918.jpg
✅ Guardado: /content/drive/MyDrive/Eigenfaces70x70/Noelia R/1742907849918_face0.jpg
Archivo: IMG_4184.JPG
✅ Guardado: /content/drive/MyDrive/Eigenfaces70x70/Noelia R/IMG_4184_face0.jpg
Archivo: IMG_20250321_140824265.jpg
✅ Guardado: /co

KeyboardInterrupt: 